**This Notebook performs data cleaning operations on a global work-from-home survey dataset in preparation for supervised machine learning**

Dee Weinacht (c) 2023

Data sourced from the Global Survey of Working Arrangements (G-SWA), used under the Creative Commons Attribution 4.0 International License:
Aksoy, Cevat Giray, Jose Maria Barrero, Nicholas Bloom, Steven J. Davis, Mathias Dolls and Pablo Zarate, 2022. “Working from Home Around the World,” Brookings Papers on Economic Activity.
https://wfhresearch.com/

---

## Initial Setup

In [1]:
#  imports
import pandas as pd
import numpy as np

Load dataset and display summary information:

In [2]:
df = pd.read_excel('G-SWA.xlsx')
df.head()

,idnum,original_country,wave,country,originalcountry,regionname,gender,agegroups,education,industry_job,...,return_office,ever_WFH,commute_time_hs,deaths_pc,LSI,reg_deaths_pc,subn_LSI,oxf_LSI,gdppc2019,mask
0,1,Australia,1,Australia 1,27,Victoria,Male,50-59,Secondary,Professional & Business Services,...,NaN,100,0.200000,0.000036,11.689555,0.000123,9.047081,1049.078321,58781.046657,7.840399
1,2,Australia,1,Australia 1,27,Queensland,Male,40-49,Tertiary,Professional & Business Services,...,NaN,100,1.500000,0.000036,11.689555,0.000001,7.829032,1049.078321,58781.046657,7.840399
2,3,Australia,1,Australia 1,27,Victoria,Male,30-39,Graduate,Professional & Business Services,...,NaN,100,2.000000,0.000036,11.689555,0.000123,9.047081,1049.078321,58781.046657,7.840399
3,4,Australia,1,Australia 1,27,Victoria,Male,40-49,Graduate,Health Care & Social Assistance,...,NaN,0,1.000000,0.000036,11.689555,0.000123,9.047081,1049.078321,58781.046657,7.840399
4,5,Australia,1,Australia 1,27,Victoria,Male,20-29,Secondary,Other,...,NaN,0,0.033333,0.000036,11.689555,0.000123,9.047081,1049.078321,58781.046657,7.840399


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36078 entries, 0 to 36077
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   idnum                   36078 non-null  int64  
 1   original_country        36078 non-null  object 
 2   wave                    36078 non-null  int64  
 3   country                 36078 non-null  object 
 4   originalcountry         36078 non-null  int64  
 5   regionname              7849 non-null   object 
 6   gender                  36078 non-null  object 
 7   agegroups               36078 non-null  object 
 8   education               36078 non-null  object 
 9   industry_job            36078 non-null  object 
 10  age                     36078 non-null  int64  
 11  tertiary                36078 non-null  int64  
 12  graduate                36078 non-null  int64  
 13  married                 26745 non-null  float64
 14  male                    36078 non-null

The original dataset is comprised of 36,078 entries with 34 variables.

## Dataset Reduction

The 'return_office' variable will be the target for supervised machine learning. Therefore, the dataset is reduced to only rows with a response for 'return_office' and who have ever primarily worked from home:

In [4]:
df = df.dropna(subset='return_office')
df = df[df['ever_WFH'] == 100]
len(df)

12474

Next, columns that contain information irrelevant to the machine learning task will be dropped:

In [5]:
df.drop(labels=['idnum', 'country', 'wave', 'ever_WFH'], axis=1, inplace=True, errors='ignore')
len(df.columns)

30

Next, list the variables with missing values with how many entries they contain:

In [6]:
for col in df:
    if df[col].count() < len(df):
        print(col, df[col].count())

regionname 3323
married 7757
with_kids 7752
male_with_kids 7752
n_work_home 11453
daysemployer_work_home 11988
commute_time_hs 12454
reg_deaths_pc 3236
subn_LSI 3236


Drop the variables with fewer than 4000 entries (~1/3 of total):

In [7]:
df.dropna(axis=1, thresh=4000, inplace=True)
for col in df:
    if df[col].count() < len(df):
        print(col, df[col].count())

married 7757
with_kids 7752
male_with_kids 7752
n_work_home 11453
daysemployer_work_home 11988
commute_time_hs 12454


There are 6 remaining variables with some missing values. The missing values will be addressed later.

List all column names to identify possible redundant variables:

In [8]:
for num, col in enumerate(df.columns):
    print(num+1, col)

1 original_country
2 originalcountry
3 gender
4 agegroups
5 education
6 industry_job
7 age
8 tertiary
9 graduate
10 married
11 male
12 with_kids
13 male_with_kids
14 n_work_home
15 daysemployer_work_home
16 daysemployee_work_home
17 value_WFH_rawpercent25
18 commuting_time
19 WFH_expectations1
20 WFHperceptions
21 return_office
22 commute_time_hs
23 deaths_pc
24 LSI
25 oxf_LSI
26 gdppc2019
27 mask


Identify redundant variables to remove:

In [9]:
df['original_country'].value_counts()

India          836
Malaysia       808
Singapore      801
Netherlands    695
China          649
Turkey         629
Australia      578
Russia         526
UK             509
Canada         501
Ukraine        496
Sweden         493
Germany        488
USA            480
Greece         450
Italy          438
Korea          410
Spain          403
Taiwan         400
Brazil         347
Poland         331
Hungary        327
France         309
Austria        301
Japan          269
Name: original_country, dtype: int64

In [10]:
df['originalcountry'].value_counts()

19    836
22    808
24    801
10    695
18    649
15    629
27    578
23    526
16    509
2     501
17    496
14    493
6     488
3     480
7     450
9     438
21    410
13    403
25    400
1     347
11    331
8     327
5     309
4     301
20    269
Name: originalcountry, dtype: int64

In [11]:
df['commuting_time'].corr(df['commute_time_hs'])

1.0

In [12]:
df['gender'].value_counts()

Female                     6519
Male                       5935
Other/Prefer not to say      20
Name: gender, dtype: int64

In [13]:
df['male'].value_counts()

0    6539
1    5935
Name: male, dtype: int64

In [14]:
df['with_kids'].value_counts()

0.0    4044
1.0    3708
Name: with_kids, dtype: int64

In [15]:
df['male_with_kids'].value_counts()

0.0    5947
1.0    1805
Name: male_with_kids, dtype: int64

In [16]:
len(df[(df['male'] == 1) & (df['with_kids'] == 1)])

1805

In [17]:
df['agegroups'].values

array(['40-49', '30-39', '20-29', ..., '20-29', '40-49', '20-29'],
      dtype=object)

In [18]:
df['age'].values

array([44, 32, 21, ..., 27, 43, 26], dtype=int64)

In [19]:
df['education'].value_counts()

Graduate     5519
Tertiary     4466
Secondary    2489
Name: education, dtype: int64

- Countries are listed by both number and name
- 'commuting time' and 'commute_time_hs' convey the same information, but with different units.
- 'male' is a dummy variable for 'gender' but excludes 'Other/Prefer not to say'
- 'male_with_kids' is redundant with 'gender' and 'with_kids'
- 'agegroups' is just 'age' assigned to groupings
- 'tertiary' and 'graduate' are dummy variables for 'education'

Redundant columns are dropped from the dataset:

In [20]:
df.drop(labels=['originalcountry', 'commuting_time', 'male', 'male_with_kids', 'agegroups', 'tertiary', 'graduate'], axis=1, inplace=True, errors='ignore')
for num, col in enumerate(df.columns):
    print(num+1, col)

1 original_country
2 gender
3 education
4 industry_job
5 age
6 married
7 with_kids
8 n_work_home
9 daysemployer_work_home
10 daysemployee_work_home
11 value_WFH_rawpercent25
12 WFH_expectations1
13 WFHperceptions
14 return_office
15 commute_time_hs
16 deaths_pc
17 LSI
18 oxf_LSI
19 gdppc2019
20 mask


There are 20 remaining variables in the dataset.

## Fix Structural Errors

Rename columns with a consistent naming scheme:

In [21]:
df.rename({'n_work_home': 'work_home_days_current', 'daysemployer_work_home': 'work_home_days_employer', 'daysemployee_work_home': 'work_home_days_employee', 'value_WFH_rawpercent25': 'WFH_value', 'WFH_expectations1': 'WFH_expectation', 'WFHperceptions': 'WFH_perception', 'gdppc2019': 'GDP_pc_2019'}, axis=1, inplace=True) 

## Fill missing values

List columns missing values: 

In [22]:
for col in df:
    if df[col].count() < len(df):
        print(col, df[col].count())

married 7757
with_kids 7752
work_home_days_current 11453
work_home_days_employer 11988
commute_time_hs 12454


Fill missing values by using a different correlated variable to group by and take the central value of the group:

Address the 'married variable:

In [23]:
df.corrwith(df['married'])

age                        0.161358
married                    1.000000
with_kids                  0.380384
work_home_days_current    -0.031746
work_home_days_employer    0.015895
work_home_days_employee   -0.022241
WFH_value                  0.038750
WFH_expectation            0.010749
WFH_perception             0.000043
commute_time_hs            0.006638
deaths_pc                 -0.043830
LSI                       -0.037766
oxf_LSI                   -0.013507
GDP_pc_2019               -0.075406
mask                      -0.029473
dtype: float64

In [24]:
df['married'] = df['married'].fillna(df.groupby('age')['married'].transform('median'))

In [25]:
len(df[df['married'].notnull() == False])

0

There are 0 remaining null values for the 'married' variable.

Next, address the 'with_kids' variable:

In [26]:
df.corrwith(df['with_kids'])

age                       -0.136069
married                    0.380384
with_kids                  1.000000
work_home_days_current    -0.030892
work_home_days_employer    0.059927
work_home_days_employee   -0.038923
WFH_value                  0.084070
WFH_expectation            0.039903
WFH_perception             0.035732
commute_time_hs            0.030443
deaths_pc                 -0.143992
LSI                        0.042593
oxf_LSI                    0.104853
GDP_pc_2019               -0.181807
mask                       0.078538
dtype: float64

In [27]:
df['with_kids'] = df['with_kids'].fillna(df.groupby('age')['with_kids'].transform('median'))
len(df[df['with_kids'].notnull() == False])

0

There are 0 remaining null values for the 'with_kids' variable.

Next, address the 'work_home_days_employer' variable:

In [28]:
df.corrwith(df['work_home_days_employer'])

age                       -0.044307
married                   -0.007388
with_kids                  0.049631
work_home_days_current     0.426000
work_home_days_employer    1.000000
work_home_days_employee    0.479360
WFH_value                  0.142905
WFH_expectation            0.207597
WFH_perception             0.175806
commute_time_hs            0.059209
deaths_pc                 -0.022329
LSI                        0.095135
oxf_LSI                    0.065029
GDP_pc_2019                0.052894
mask                       0.066025
dtype: float64

In [29]:
df['work_home_days_employer'] = df['work_home_days_employer'].fillna(df.groupby('work_home_days_employee')['work_home_days_employer'].transform('median'))
len(df[df['work_home_days_employer'].notnull() == False])

0

There are 0 remaining null values for the 'work_home_days_employer' variable.

Next, address the 'work_home_days_current' variable:

In [30]:
df.corrwith(df['work_home_days_current'])

age                        0.018524
married                   -0.025248
with_kids                 -0.007943
work_home_days_current     1.000000
work_home_days_employer    0.433804
work_home_days_employee    0.459371
WFH_value                  0.124702
WFH_expectation            0.215371
WFH_perception             0.166153
commute_time_hs            0.088390
deaths_pc                 -0.022779
LSI                        0.102137
oxf_LSI                    0.082690
GDP_pc_2019                0.123212
mask                       0.067546
dtype: float64

In [31]:
df['work_home_days_current'] = df['work_home_days_current'].fillna(df.groupby('work_home_days_employee')['work_home_days_current'].transform('median'))
len(df[df['work_home_days_current'].notnull() == False])

0

There are 0 remaining null values for the 'work_home_days_current' variable.

Next, address the 'WFH_expectation' variable:

In [32]:
df.corrwith(df['WFH_expectation'])

age                       -0.089833
married                   -0.016588
with_kids                  0.039608
work_home_days_current     0.221209
work_home_days_employer    0.212086
work_home_days_employee    0.356367
WFH_value                  0.339381
WFH_expectation            1.000000
WFH_perception             0.476761
commute_time_hs            0.114559
deaths_pc                  0.023870
LSI                        0.073299
oxf_LSI                    0.076404
GDP_pc_2019                0.006743
mask                       0.074056
dtype: float64

In [33]:
df['WFH_expectation'] = df['WFH_expectation'].fillna(df.groupby('WFH_perception')['WFH_expectation'].transform('median'))
len(df[df['WFH_expectation'].notnull() == False])

0

There are 0 remaining null values for the 'WFH_expectation' variable.

Next, address the 'commute_time_hs' variable:

In [34]:
df.corrwith(df['commute_time_hs'])

age                       -0.015528
married                    0.008993
with_kids                  0.031346
work_home_days_current     0.089488
work_home_days_employer    0.049030
work_home_days_employee    0.105552
WFH_value                  0.058915
WFH_expectation            0.114559
WFH_perception             0.102577
commute_time_hs            1.000000
deaths_pc                 -0.126848
LSI                        0.006921
oxf_LSI                   -0.008126
GDP_pc_2019               -0.027269
mask                       0.054722
dtype: float64

In [36]:
df['commute_time_hs'] = df['commute_time_hs'].fillna(df.groupby('original_country')['commute_time_hs'].transform('mean'))
len(df[df['commute_time_hs'].notnull() == False])

0

There are 0 remaining null values for the 'commute_time_hs' variable.

## Final Check of Clean Data

In [37]:
df.head()

,original_country,gender,education,industry_job,age,married,with_kids,work_home_days_current,work_home_days_employer,work_home_days_employee,WFH_value,WFH_expectation,WFH_perception,return_office,commute_time_hs,deaths_pc,LSI,oxf_LSI,GDP_pc_2019,mask
575,Australia,Female,Graduate,Education,44,1.0,1.0,5.0,0.0,2,2.5,5.0,70,Look for a job to WFH 1-2 days,0.666667,0.000149,16.855146,1441.499009,58781.046657,13.840399
578,Australia,Female,Tertiary,Education,32,1.0,1.0,5.0,0.0,4,-30.0,0.0,0,Quit job,0.366667,0.000149,16.855146,1441.499009,58781.046657,13.840399
580,Australia,Male,Secondary,Retail Trade,21,0.0,0.0,5.0,5.0,5,7.5,25.0,95,Comply and return to worksite,1.000000,0.000149,16.855146,1441.499009,58781.046657,13.840399
581,Australia,Male,Graduate,Information,43,1.0,1.0,3.0,5.0,4,12.5,15.0,0,Comply and return to worksite,1.416667,0.000149,16.855146,1441.499009,58781.046657,13.840399
582,Australia,Female,Graduate,Education,41,1.0,1.0,3.0,3.0,3,12.5,15.0,70,Comply and return to worksite,2.183333,0.000149,16.855146,1441.499009,58781.046657,13.840399


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12474 entries, 575 to 36076
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   original_country         12474 non-null  object 
 1   gender                   12474 non-null  object 
 2   education                12474 non-null  object 
 3   industry_job             12474 non-null  object 
 4   age                      12474 non-null  int64  
 5   married                  12474 non-null  float64
 6   with_kids                12474 non-null  float64
 7   work_home_days_current   12474 non-null  float64
 8   work_home_days_employer  12474 non-null  float64
 9   work_home_days_employee  12474 non-null  int64  
 10  WFH_value                12474 non-null  float64
 11  WFH_expectation          12474 non-null  float64
 12  WFH_perception           12474 non-null  int64  
 13  return_office            12474 non-null  object 
 14  commute_time_hs     

In [39]:
df.describe()

,age,married,with_kids,work_home_days_current,work_home_days_employer,work_home_days_employee,WFH_value,WFH_expectation,WFH_perception,commute_time_hs,deaths_pc,LSI,oxf_LSI,GDP_pc_2019,mask
count,12474.000000,12474.000000,12474.000000,12474.000000,12474.000000,12474.000000,12474.000000,12474.000000,12474.000000,12474.000000,12474.000000,12474.000000,12474.000000,12474.000000,12474.000000
mean,39.000561,0.779622,0.495310,2.622415,1.452862,2.503447,7.611432,7.267917,47.255491,1.207783,0.001360,17.649947,1405.698204,30269.132251,13.323986
std,10.161728,0.414518,0.498251,2.089775,1.752797,1.758177,10.335732,12.015619,42.326908,0.903702,0.001053,4.500118,204.058889,20137.492244,7.118360
min,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-30.000000,-25.000000,-95.000000,0.000000,0.000003,5.005645,760.270925,1972.757821,0.000000
25%,31.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,25.000000,0.583333,0.000356,16.104570,1239.395000,11414.578704,7.155837
50%,38.000000,1.000000,0.000000,3.000000,0.000000,2.000000,7.500000,5.000000,70.000000,1.000000,0.001244,18.474462,1441.499009,31674.311518,15.566667
75%,48.000000,1.000000,1.000000,5.000000,3.000000,4.000000,12.500000,15.000000,70.000000,1.583333,0.002266,21.491935,1529.053860,47750.879662,19.800000
max,59.000000,1.000000,1.000000,5.000000,5.000000,5.000000,30.000000,25.000000,95.000000,8.000000,0.004298,22.879032,1688.480806,61173.904769,21.666667


In [40]:
df.corr()

,age,married,with_kids,work_home_days_current,work_home_days_employer,work_home_days_employee,WFH_value,WFH_expectation,WFH_perception,commute_time_hs,deaths_pc,LSI,oxf_LSI,GDP_pc_2019,mask
age,1.000000,0.235740,-0.166518,0.019212,-0.035434,-0.001034,-0.090054,-0.089833,-0.070701,-0.015511,0.030877,-0.078248,-0.071523,0.164207,-0.037017
married,0.235740,1.000000,0.343778,-0.023877,-0.009719,-0.022114,-0.009145,-0.016588,-0.023760,0.009073,-0.099541,-0.080065,-0.052362,-0.016695,0.043107
with_kids,-0.166518,0.343778,1.000000,-0.004511,0.041769,-0.008470,0.057217,0.039608,0.031152,0.031270,-0.105925,0.012638,0.028871,-0.111597,0.070429
work_home_days_current,0.019212,-0.023877,-0.004511,1.000000,0.433790,0.499916,0.137198,0.221209,0.167721,0.089439,-0.017226,0.098730,0.076224,0.117000,0.069425
work_home_days_employer,-0.035434,-0.009719,0.041769,0.433790,1.000000,0.496249,0.146351,0.212086,0.173901,0.048931,-0.012693,0.093416,0.062746,0.051316,0.065067
work_home_days_employee,-0.001034,-0.022114,-0.008470,0.499916,0.496249,1.000000,0.282649,0.356367,0.261954,0.105438,0.051174,0.064978,0.017641,0.094111,0.078577
WFH_value,-0.090054,-0.009145,0.057217,0.137198,0.146351,0.282649,1.000000,0.339381,0.272978,0.058885,0.053995,0.067798,0.076567,-0.064655,0.066862
WFH_expectation,-0.089833,-0.016588,0.039608,0.221209,0.212086,0.356367,0.339381,1.000000,0.476761,0.114433,0.023870,0.073299,0.076404,0.006743,0.074056
WFH_perception,-0.070701,-0.023760,0.031152,0.167721,0.173901,0.261954,0.272978,0.476761,1.000000,0.102582,-0.034730,0.056873,0.056559,0.050817,0.067476
commute_time_hs,-0.015511,0.009073,0.031270,0.089439,0.048931,0.105438,0.058885,0.114433,0.102582,1.000000,-0.126996,0.006663,-0.008352,-0.027208,0.054576


The cleansed dataset contains 12,474 entries with 20 variables. There are no remaining null values or redundant variables.

Save clean data to a new excel document:

In [41]:
df.to_excel('G-SWA Clean.xlsx', index=False)